<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/openbbPlatformAsLLMTools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenBB Tools

## Getting Started

### Install dependencies
####https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/orchestration/langchain/intro_langchain_palm_api.ipynb#scrollTo=eVpPcvsrkzCk

In [ ]:
!pip install openbb
!pip install openbb-charting

!pip install pydantic

!pip install langchain
!pip install langchain_community
#!pip install langchain_openai
!pip install langchain-core
!pip install langchain-google-vertexai
!pip install -U langchain-google-genai
# need to restart runtime after installing pydantic

### Set the VertexAI Key

In [ ]:
import google.generativeai as genai
from google.api_core import retry
from google.colab import userdata
gkey = userdata.get('AI_STUDIO_KEY')
pat = userdata.get('PAT')
genai.configure(api_key=gkey)
import vertexai
from google.colab import auth
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
    ToolConfig
)
PROJECT_ID = "datascience-projects"  # @param {type:"string"}
# Set the project id
#! gcloud config set project {PROJECT_ID}
#auth.authenticate_user()
vertexai.init(
    project=PROJECT_ID,
    location="us-central1",
    staging_bucket="gs://mm_dataflow_bucket",
)
model = GenerativeModel(model_name="gemini-1.5-pro-001")

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

## Testing Vertex AI Custom function api

In [ ]:
function_name = "get_current_weather"
get_current_weather_func = FunctionDeclaration(
    name=function_name,
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "The city name of the location for which to get the weather."}},
    },
)

In [ ]:
# Define the user's prompt in a Content object that we can reuse in model calls
user_prompt_content = Content(
    role="user",
    parts=[
        Part.from_text("What is the weather like in Boston?"),
    ],
)

In [ ]:
# Define a tool that includes the function declaration get_current_weather_func
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

# Send the prompt and instruct the model to generate content using the Tool object that you just created
response = model.generate_content(
    user_prompt_content,
    generation_config={"temperature": 0},
    tools=[weather_tool],
    tool_config=ToolConfig(
        function_calling_config=ToolConfig.FunctionCallingConfig(
            # ANY mode forces the model to predict a function call
            mode=ToolConfig.FunctionCallingConfig.Mode.ANY,
            # Allowed functions to call when the mode is ANY. If empty, any one of
            # the provided functions are called.
            allowed_function_names=["get_current_weather"],
    ))
)
response_function_call_content = response.candidates[0].content
print(response_function_call_content)

In [ ]:
# Check the function name that the model responded with, and make an API call to an external system
if (
    response.candidates[0].content.parts[0].function_call.name
    == "get_current_weather"
):
    # Extract the arguments to use in your API call
    location = (
        response.candidates[0].content.parts[0].function_call.args["location"]
    )

    # Here you can use your preferred method to make an API request to fetch the current weather, for example:
    # api_response = requests.post(weather_api_url, data={"location": location})

    # In this example, we'll use synthetic data to simulate a response payload from an external API
    api_response = """{ "location": "Boston, MA", "temperature": 38, "description": "Partly Cloudy",
                    "icon": "partly-cloudy", "humidity": 65, "wind": { "speed": 10, "direction": "NW" } }"""

In [ ]:
response = model.generate_content(
    [
        user_prompt_content,  # User prompt
        response_function_call_content,  # Function call response
        Content(
            parts=[
                Part.from_function_response(
                    name="get_current_weather",
                    response={
                        "content": api_response,  # Return the API response to Gemini
                    },
                )
            ],
        ),
    ],
    tools=[weather_tool],
)
# Get the model summary response
summary = response.candidates[0].content.parts[0].text
print(summary)

### Import packages

In [ ]:
from openbb import obb

### Login

In [ ]:
obb.account.login(pat=pat)

## OpenBB tool pre-requisites

### Callable

In [ ]:
obb.equity.price.historical(symbol="AAPL", provider="fmp").to_df()

In [ ]:
obb.equity.price.historical("AAPL", start_date="2022-01-01", provider='fmp')

### Input Schema

In [ ]:
obb.coverage.command_model[".equity.price.historical"]["openbb"]["QueryParams"]

In [ ]:
obb.coverage.command_model[".equity.price.historical"]["polygon"]["QueryParams"]

### Documentation

In [ ]:
help(obb.equity.price.historical)

## OpenBB tool

In [ ]:
from langchain_core.tools import StructuredTool

llm_historical_price = StructuredTool.from_function(
    func=obb.equity.price.historical,
    description=obb.equity.price.historical.__doc__.split('\n')[0]  # Use first line of docstring
)

## Multiple OpenBB Tools

In [ ]:
llm_tools = [
    StructuredTool.from_function(
        name=name,
        func=schema['callable'],
        description=schema['callable'].__doc__.split('\n')[0]
    )
    for name, schema
    in obb.coverage.command_schemas().items()
]

## OpenBB Tools fed to agent

In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent, create_tool_calling_agent
from langchain_google_vertexai import VertexAI, ChatVertexAI #, ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a very powerful assistant, but don't know current events"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

llm_tools = [
    StructuredTool.from_function(
        func=obb.equity.price.quote,
        description=obb.equity.price.quote.__doc__.split("\n")[0]
    )
]

#llm = VertexAI(model_name="gemini-pro")
llm = ChatVertexAI(model="gemini-pro")


In [ ]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.chat_models import ChatVertexAI
from google.cloud import aiplatform
import time
from typing import List

# LangChain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI

print(f"Vertex AI SDK version: {aiplatform.__version__}")

In [ ]:
agent = create_tool_calling_agent(llm=llm, tools=llm_tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=llm_tools, verbose=True)

agent_executor.invoke({"input": "What is the latest stock price of AAPL?"})